In [1]:
# Associate words with archetypes/character traits as intermediate layer
# and with influencer as the "last" layer

# Dependencies
import pandas as pd
import dask.dataframe as dd
import numpy as np
from tqdm.notebook import trange, tqdm
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter
import pickle

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

In [2]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl_new.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [3]:
# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [4]:
# Split dataset into train/test, in 75-25% proportion
train_df = pd.read_csv("agds_structures/train_90.csv", index_col=0)
test_df = pd.read_csv("agds_structures/test_10.csv", index_col=0)

BASE_DIR = "instagram_cleared"

def generate_dataset(dataset_frame):
    posts = []

    # Iterate over whole DataFrame
    for i, row in tqdm(dataset_frame.iterrows()):
        profile_posts = []
        profile_hashtags = []

        # Get all posts per profile
        profile_path = os.path.join(BASE_DIR, i)
        for file in os.listdir(profile_path):
            if not file.endswith(".toml"):
                with open(os.path.join(profile_path, file), "r") as post_f:
                    read_text = post_f.read()
                    profile_posts.append(remove_stopwords(clean_up_text(read_text)))
                    profile_hashtags.append(extract_hashtags(read_text))

        # Merge lists - a single list for a single influencer
        profile_hashtags = list(itertools.chain.from_iterable(profile_hashtags))
        posts.append(list(itertools.chain.from_iterable([profile_posts, [profile_hashtags]])))
    return posts

train_posts = generate_dataset(train_df)
test_posts = generate_dataset(test_df)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [5]:
# Map training usernames to indices
users = list(train_df.index.values)
user_indices = {k: users.index(k) for k in users}

In [6]:
# Load softmax AGDS model
with open("agds_structures/finetuned_s90_10_word_trait_array.pickle", "rb") as f:
    softmax_word_df = pickle.load(f)

In [7]:
# AGDS - discrete approach

def merge_dicts(dict_a, dict_b) -> dict:
    out_dict = dict_a
    for k, v in dict_b.items():
        if k in out_dict.keys():
            out_dict[k] += v
        else:
            out_dict[k] = v
    return out_dict

def min_max_normalize(df_to_normalize):
    normalized_df = df_to_normalize
    cols = normalized_df.columns.tolist()
    for col in tqdm(cols, position=1):
        normalized_df[col] = (normalized_df[col] - normalized_df[col].min()) / (normalized_df[col].max() - normalized_df[col].min())
    return normalized_df
    

# Iterate over all of the traits/archetypes
word_set = set()
trait_df_list = []
for trait in tqdm(trait_list, position=0):
    # Select influencers which have the given archetype annotated
    subset_df = train_df[trait]
    trait_subframe = pd.DataFrame()
    for trait_class in range(5):
        class_df = subset_df.loc[subset_df == trait_class]
        subset_indices = [user_indices[idx] for idx in class_df.index.values]

        # Get all posts for the list of influencers
        f = operator.itemgetter(*subset_indices)
        sublist = list(f(train_posts))

        # Counter to calculate each word occurrences
        trait_total = 0
        out_dict = {}
        for i, post_set in enumerate(sublist):
            trait_ctr = Counter(itertools.chain.from_iterable(post_set))
            trait_total += sum(trait_ctr.values())
            out_dict = merge_dicts(out_dict, trait_ctr)
        out_dict = {k: float(v / trait_total) for k, v in out_dict.items()}
        word_set.update(out_dict.keys())
        trait_ctr = {trait_class: out_dict}
        trait_tmp_df = pd.DataFrame.from_dict(trait_ctr, orient="index")
        trait_subframe = trait_subframe.append(trait_tmp_df)

    # Append the new dataframe
    #word_df = word_df.append(trait_subframe)
    
    trait_subframe = min_max_normalize(trait_subframe)
    trait_df_list.append(trait_subframe)
    
softmax_word_df = pd.concat(trait_df_list, keys=trait_list)

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

...

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

  0%|          | 0/211596 [00:00<?, ?it/s]

In [8]:
# Check the calculation results
softmax_word_df

good   morning      wish       you      nice  relaxing  \
innocent    0  1.000000  1.000000  1.000000  0.985589  1.000000  1.000000   
            1  0.164918  0.363648  0.005689  1.000000  0.000000       NaN   
            2  0.059452  0.032827  0.000000  0.885980  0.026943  0.003168   
            3  0.053402  0.063776  0.003512  0.250463  0.024454  0.000000   
            4  0.000000  0.000000       NaN  0.000000  0.013889       NaN   
...                 ...       ...       ...       ...       ...       ...   
allocentric 0  0.661119  1.000000       NaN  0.689478  0.441220       NaN   
            1  1.000000  0.443374  1.000000  0.659732  1.000000  1.000000   
            2  0.028338  0.171131  0.000000  0.000000  0.000000       NaN   
            3  0.000000  0.078641  0.060778  0.162895  0.168929       NaN   
            4  0.213404  0.000000  0.041266  1.000000  0.086842  0.000000   

               thursday  saturday    friday     start  ...  #używane  \
innocent    0  1.000000  1.000000  0.579753  1.000000  ...       NaN   
            1       NaN       NaN  1.000000  0.437572  ...       NaN   
            2  0.000000  0.000000  0.097603  0.325299  ...       NaN   
            3  0.021592  0.062854  0.216949  0.151777  ...       NaN   
            4       NaN       NaN  0.000000  0.000000  ...       NaN   
...                 ...       ...       ...       ...  ...       ...   
allocentric 0       NaN  1.000000  0.568315  0.000000  ...       NaN   
            1  1.000000  0.485613  0.956797  0.605759  ...       NaN   
            2       NaN       NaN  0.000000  0.113663  ...       NaN   
            3       NaN  0.139209  1.000000  0.426277  ...       NaN   
            4  0.000000  0.000000  0.410081  1.000000  ...       NaN   

               #podzielnia  #ekobiuro  #goHi2020  #hackaton  #jachranka  \
innocent    0          NaN        NaN        NaN        NaN         NaN   
            1          NaN        NaN        NaN        NaN         NaN   
            2          NaN        NaN        NaN        NaN         NaN   
            3          NaN        NaN        NaN        NaN         NaN   
            4          NaN        NaN        NaN        NaN         NaN   
...                    ...        ...        ...        ...         ...   
allocentric 0          NaN        NaN        NaN        NaN         NaN   
            1          NaN        NaN        NaN        NaN         NaN   
            2          NaN        NaN        NaN        NaN         NaN   
            3          NaN        NaN        NaN        NaN         NaN   
            4          NaN        NaN        NaN        NaN         NaN   

               #greenladies  #greenguys  #polishheroes  #bestgifts  
innocent    0           NaN         NaN            NaN         NaN  
            1           NaN         NaN            NaN         NaN  
            2           NaN         NaN            NaN         NaN  
            3           NaN         NaN            NaN         NaN  
            4           NaN         NaN            NaN         NaN  
...                     ...         ...            ...         ...  
allocentric 0           NaN         NaN            NaN         NaN  
            1           NaN         NaN            NaN         NaN  
            2           NaN         NaN            NaN         NaN  
            3           NaN         NaN            NaN         NaN  
            4           NaN         NaN            NaN         NaN  

[185 rows x 211596 columns]

In [9]:
# Fill NaN values with 0
softmax_word_df = softmax_word_df.fillna(0)

softmax_word_df.to_pickle("normalized_s90_10_word_trait_array.pickle")

In [7]:
# Create word map for softmax structure
softmax_word_map = softmax_word_df.columns.tolist()

In [8]:
from scipy.special import softmax

def get_trait_dot_product(post_text: str, word_map: list, word_dataframe: pd.DataFrame) -> list:
    # Filter out the text
    filtered_post = remove_stopwords(clean_up_text(post_text))
    filtered_post += extract_hashtags(post_text)
    
    # Create a vector for dot product vector
    post_vector = [0] * len(word_map)
    
    # Calculate word occurrences
    word_ctr = Counter(filtered_post)
    
    for word, freq in word_ctr.items():
        if word in word_map:
            post_vector[word_map.index(word)] = freq
    
    # Calculate dot product for a given text
    word_dot = word_dataframe.dot(post_vector)
    
    out_vec = pd.Series()
    for trait in trait_list:
        out_vec = out_vec.append(pd.Series([np.argmax(softmax(word_dot.loc[trait]))], index=[trait]))
    
    return out_vec

# Trait accuracy - round the results
def natural_round(x: float) -> int:
    out = int(x // 1)
    return out + 1 if (x - out) >= 0.5 else out

def accuracy_per_trait(input_vector: pd.Series, annotated_vector: pd.Series) -> np.array:
    out_array = np.array([0] * 37, dtype=np.int)
    for i in range(len(out_array)):
        if input_vector[i] == annotated_vector[i]:
            out_array[i] = 1
    return out_array

In [9]:
pbar = tqdm(train_df.iterrows())
accuracy = 0

# Out accuracy vector
total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(train_posts[users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, softmax_word_map, softmax_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(total_accuracy, users.index(idx)+1))*100, 2)}")

0it [00:00, ?it/s]

<ipython-input-8-4ffef00e153a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out_vec = pd.Series()


In [10]:
# Show total accuracy
scaled_train_accuracy = np.divide(total_accuracy, len(train_df))
avg_train_accuracy = np.mean(scaled_train_accuracy)

print("--- ACCURACY ON TRAINING DATASET ---")

print(f"Average train dataset accuracy: {round(avg_train_accuracy*100, 2)}%")
print("Accuracy per trait:")
for i in range(len(trait_list)):
    print(f"{trait_list[i]}: {round(scaled_train_accuracy[i] * 100, 2)}%")

--- ACCURACY ON TRAINING DATASET ---
Average train dataset accuracy: 88.83%
Accuracy per trait:
innocent: 90.75%
sage: 85.88%
explorer: 85.39%
outlaw: 88.15%
magician: 88.47%
hero: 90.1%
lover: 86.2%
jester: 90.26%
everyman: 89.29%
caregiver: 84.09%
ruler: 83.6%
creator: 87.01%
dominant: 90.1%
submissive: 89.12%
maximalist: 85.06%
minimalist: 91.56%
inspiring: 93.18%
systematic: 87.82%
discovering: 93.83%
conservative: 90.26%
verifying: 91.72%
overlooking: 89.12%
sharpening: 91.23%
harmonic: 91.88%
empathic: 91.07%
matter_of_fact: 82.79%
brave: 88.31%
protective: 87.66%
generous: 92.37%
thrifty: 87.99%
favourable: 89.29%
balanced: 92.37%
sensuality: 89.94%
intelligent: 86.53%
believe: 91.72%
egocentric: 87.01%
allocentric: 85.55%


In [11]:
# Set up environment for test dataset
test_users = list(test_df.index.values)
test_user_indices = {k: test_users.index(k) for k in test_users}

In [15]:
pbar = tqdm(test_df.iterrows())
accuracy = 0

# Out accuracy vector
test_total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(test_posts[test_users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, softmax_word_map, softmax_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    test_total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(test_total_accuracy, test_users.index(idx)+1))*100, 2)}")

0it [00:00, ?it/s]

<ipython-input-8-4ffef00e153a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out_vec = pd.Series()


In [16]:
# Show total accuracy
scaled_test_accuracy = np.divide(test_total_accuracy, len(test_df))
avg_test_accuracy = np.mean(scaled_test_accuracy)

print("--- ACCURACY ON TESTING DATASET ---")

print(f"Average test dataset accuracy: {round(avg_test_accuracy*100, 2)}%")
print("Accuracy per trait:")
for i in range(len(trait_list)):
    print(f"{trait_list[i]}: {round(scaled_test_accuracy[i] * 100, 2)}%")

--- ACCURACY ON TESTING DATASET ---
Average test dataset accuracy: 91.19%
Accuracy per trait:
innocent: 91.3%
sage: 84.06%
explorer: 92.75%
outlaw: 94.2%
magician: 91.3%
hero: 95.65%
lover: 95.65%
jester: 94.2%
everyman: 88.41%
caregiver: 95.65%
ruler: 89.86%
creator: 95.65%
dominant: 89.86%
submissive: 88.41%
maximalist: 94.2%
minimalist: 91.3%
inspiring: 94.2%
systematic: 86.96%
discovering: 89.86%
conservative: 88.41%
verifying: 92.75%
overlooking: 89.86%
sharpening: 92.75%
harmonic: 94.2%
empathic: 91.3%
matter_of_fact: 81.16%
brave: 85.51%
protective: 95.65%
generous: 94.2%
thrifty: 91.3%
favourable: 85.51%
balanced: 91.3%
sensuality: 89.86%
intelligent: 89.86%
believe: 89.86%
egocentric: 94.2%
allocentric: 92.75%
